# Lógica de Primer Orden (FOL) y **kanren** en Python

## Objetivo

Este notebook explica, paso a paso y con ejemplos ejecutables, los conceptos clave de la **Lógica de Primer Orden (FOL)** y cómo experimentar con lógica relacional en Python usando la librería **kanren**.

## Estructura

1. Introducción y objetivos
2. Conceptos teóricos de FOL (sintaxis y semántica)
3. Inferencia en FOL: un panorama (unificación, resolución, CNF, Skolemización)
4. Ejemplos formales (dominio de parentesco)
5. Introducción a `kanren` — instalación y primeros pasos
6. Ejemplos prácticos con `kanren`: hechos, reglas, consultas y recursión

## 1) Requisitos previos

- Python 3.8+ (recomendado). Algunos usuarios han reportado incompatibilidades con versiones muy recientes en librerías antiguas; si ves errores relativos a `collections.Iterator` podría ser por compatibilidad con la versión de la librería. Ejecuta el ambiente en un `virtualenv` o `venv` para aislar dependencias.


Instalación (terminal):


```bash
pip install kanren
```


(En este notebook supondremos que `kanren` ya está instalado.)

In [3]:
pip install kanren

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kanren: filename=kanren-0.2.3-py3-none-any.whl size=15893 sha256=8617de739a058048d2f11ea8942d456ba96998cf9c3a167ddbc509dded93b391
  Stored in directory: c:\users\angelo ortiz\appdata\local\pip\cache\wheels\38\4f\35\e04a31cf00ecb89a3a31853a6207c9b3c44d3c3562e729fbf1
Successfully built kanren
Note: you may need to restart the kernel to use updated packages.


## 2) ¿Qué es la Lógica de Primer Orden (FOL)? — Resumen conceptual

**Idea:** FOL extiende la lógica proposicional permitiendo hablar de **predicados**, **funciones**, **constantes** y **cuantificadores** (\forall, \exists). Permite expresar propiedades sobre objetos y relaciones entre ellos.


**Elementos básicos:**
- **Constantes:** símbolos que representan objetos concretos (ej. `juan`, `marta`).
- **Variables:** símbolos que representan objetos no especificados (ej. `x`, `y`).
- **Funciones:** mapeos desde tuplas de objetos a objetos (ej. `padre_de(x)`).
- **Predicados:** propiedades o relaciones que pueden ser verdaderas/falsas sobre términos (ej. `Padre(x,y)`).
- **Átomos:** `Padre(juan, maria)` es una fórmula atómica.
- **Cuantificadores:** `\forall x P(x)` (para todo x), `\exists x P(x)` (existe x tal que P(x)).


**Sintaxis vs Semántica:**
- La sintaxis define qué expresiones son bien formadas (WFF).
- La semántica define lo que significan esas expresiones en un **interpretación** (modelo): dominio, asignación de constantes, funciones y predicados.

## 3) Inferencia en FOL — panorama rápido

- **Entailment (Consecuencia lógica):** una teoría \(T\) implica una sentencia \(\varphi\) si todas las interpretaciones que satisfacen \(T\) también satisfacen \(\varphi\).
- **Prueba automática:** se usan técnicas como **unificación**, **resolución** y conversión a **Clausal Normal Form (CNF)**.
- **Skolemización:** paso para eliminar cuantificadores existenciales introduciendo funciones de Skolem.
- **Completitud/Decidibilidad:** la FOL es **completamente axiomática en el sentido de que la resolución es semi-decidible**; la validez en FOL es indecidible en general (no hay algoritmo que decida para toda fórmula si es válida).

## 4) Ejemplo formal — dominio de parentesco (en notación FOL)

Hechos (en notación FOL):


- `Padre(juan, maria)`
- `Padre(ana, juan)`


Regla:


- `∀x, y, z (Padre(x, y) ∧ Padre(y, z) -> Abuelo(x, z))`


Consultas que queremos responder (ej.):


- ¿Quiénes son los abuelos de María? (equivalente a `∃x Abuelo(x, maria)` — pero en práctica queremos enumerar los `x` que satisfacen la relación)


La transición a un sistema programable hace la representación explícita y nos da mecanismos para enumerar soluciones (si existen).


## 5) De FOL a programación relacional: qué espera kanren

`kanren` es una librería de **programación relacional** (familia de miniKanren) que permite declarar relaciones y realizar **búsquedas declarativas**: en lugar de escribir un algoritmo imperativo, declaras reglas/hechos y preguntas "¿qué valores satisfacen estas condiciones?".


Concepto clave: **las consultas en kanren corresponden a fórmulas existenciales**. Por ejemplo, preguntar "¿existe x tal que `Padre(x, maria)`?" se expresa pidiendo soluciones para la variable lógica `x` con la meta `padre(x, 'maria')`.

## 6) Primeros pasos con `kanren` — importaciones y conceptos


- `var()` crea una variable lógica.
- `run(n, var, *goals)` ejecuta una consulta: devuelve hasta `n` soluciones (usar `0` para todas).
- `Relation()` y `facts(relation, ...)` permiten introducir hechos.
- `conde(...)` permite expresar disyunciones/conjunciones lógicas en forma de metas compuestas.
- `eq(a,b)` construye la meta que exige la igualdad entre `a` y `b`.

In [6]:
from kanren import run, eq, var

In [7]:
# Creamos una variable lógica
x = var()
# Pregunta: ¿qué x satisface x == 5 ?
print('run(1, x, eq(x, 5)) ->', run(1, x, eq(x, 5)))

run(1, x, eq(x, 5)) -> (5,)


In [8]:
# Pedir todas las soluciones (0) para x == 5
print('run(0, x, eq(x, 5)) ->', run(0, x, eq(x, 5)))

run(0, x, eq(x, 5)) -> (5,)


### 6.1) Hechos y relaciones: ejemplo de parentesco

Vamos a modelar `parent` (padre/madre), `ancestor` (ancestro) y consultas típicas.

In [9]:
from kanren import Relation, facts, run, var, conde


# Definimos la relación `parent`
parent = Relation()
# Añadimos hechos: facts(relation, tuple1, tuple2, ...)
facts(parent,
("ana", "juan"),
("juan", "maria"),
("marta", "ana"),
("pedro", "juan")
)


# Ejemplo: quiénes son los padres de 'juan'?
y = var()
print("Padres de 'juan':", run(0, y, parent(y, 'juan')))

Padres de 'juan': ('ana', 'pedro')


### 6.2) Reglas: definición de abuelo y ancestro

- Abuelo(x, z) si existe y tal que Padre(x, y) ∧ Padre(y, z).
- Ancestro se define recursivamente: Ancestro(x, z) si Padre(x, z) o (∃y Padre(x, y) ∧ Ancestro(y, z)).

Definimos la regla de abuelo (no recursiva) como una función que devuelve una meta

In [15]:
from kanren import run, var, conde, Relation, facts

# Definimos la relación parent
parent = Relation()
facts(parent,
      ("ana", "juan"),
      ("juan", "maria"),
      ("marta", "ana"),
      ("pedro", "juan")
     )

# Definimos la regla de abuelo
def ancestor(x, z):
    """Existe y tal que parent(x, y) y parent(y, z)"""
    y = var()
    return conde((parent(x, y), parent(y, z)))

# Consulta: ¿quiénes son abuelos de 'maria'?
x = var()
print("Abuelos de 'maria':", run(0, x, ancestor(x, 'maria')))


Abuelos de 'maria': ('ana', 'pedro')


### 6.3) Ejemplos de consultas frecuentes y su interpretación en FOL


- ¿Existe un ancestro de 'maria'?: run(1, x, ancestor(x, 'maria')) devuelve alguna solución -> corresponde a ∃x Ancestor(x, maria).
- ¿Quiénes son los ancestros?: run(0, x, ancestor(x, 'maria')) -> lista de todos los x que satisfacen Ancestor(x,maria).
- ¿Ana es ancestro de Maria? -> preguntar si la consulta tiene solución con x = 'ana'

In [16]:
print("¿'ana' es ancestro de 'maria'? ->", bool(run(1, x, ancestor('ana', 'maria'))))

¿'ana' es ancestro de 'maria'? -> True
